In [31]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import datetime as dt 
import matplotlib.pyplot as plt


########################################################### 
# Updated test code
data = pd.read_csv('data/stock_returns.csv', parse_dates=['Date'], index_col='Date').dropna(axis=1)
data = data["2000-01-01":].dropna(axis = 1)
target = np.random.choice(data.columns, 1)[0]
target = "SPY"
features = data.drop(columns=target).iloc[:, :]
x = features.copy()
y = data[target].copy()

print(y.name)

SPY


In [32]:
import sys 
sys.path.append('../')
from main import Analyze

# Setup Config: 

# Example configuration
stationarity_cfg = {
    'adf': {'max_diff': 5, 'significance': 0.05},
    'kpss': {'significance': 0.05},
    'pp': {'significance': 0.05},
    'structural_break': True,
    'gls': False,
    'nonlinear': True
}

causality_cfg = {
    'significance_level': 0.05,
    'max_lag': 4
}

In [33]:
ac = Analyze(verbose = False, stationarity_config=stationarity_cfg, causality_config=causality_cfg)

In [34]:
res = ac.results(x, y, decompose = False)

Granger Causality: SPY -> ORCL: 100%|██████████| 44/44 [00:01<00:00, 32.22it/s]


In [35]:
res['causality']

{'granger': [(('SPY', 'PEP'), array([1, 2, 3])),
  (('SPY', 'CVX'), array([1, 2, 3])),
  (('SPY', 'MO'), array([1])),
  (('SPY', 'INTC'), array([2, 3])),
  (('SPY', 'MU'), array([1, 2, 3])),
  (('SPY', 'LMT'), array([1, 2, 3])),
  (('SPY', 'WFC'), array([2, 3])),
  (('SPY', 'IBM'), array([1])),
  (('SPY', 'AMD'), array([1, 2, 3])),
  (('SPY', 'XOM'), array([2, 3])),
  (('SPY', 'SBUX'), array([2, 3])),
  (('SPY', 'KR'), array([2, 3])),
  (('SPY', 'BA'), array([2, 3])),
  (('SPY', 'GE'), array([2])),
  (('SPY', 'AA'), array([3])),
  (('SPY', 'DIS'), array([2, 3])),
  (('SPY', 'CSCO'), array([1, 2, 3])),
  (('SPY', 'CVS'), array([2, 3])),
  (('SPY', 'KO'), array([1, 2, 3])),
  (('SPY', 'OXY'), array([2, 3])),
  (('SPY', 'EOG'), array([3])),
  (('SPY', 'CLX'), array([1, 2, 3])),
  (('SPY', 'JPM'), array([2, 3])),
  (('SPY', 'K'), array([2, 3])),
  (('SPY', 'PGR'), array([2])),
  (('SPY', 'AMAT'), array([1, 2, 3]))],
 'instantaneous': [('SPY', 'PEP'),
  ('SPY', 'MMM'),
  ('SPY', 'WMT'),
  (

In [80]:
m = res['causality']['impulse_response']

In [81]:
# m.plot(impulse='SPY', response='AAPL', figsize=(10, 5))
# plt.show()

In [97]:
res.keys()
colnames = res['new_data'].columns.to_list()
cols = [(i, j) for i, j in enumerate(colnames)]


In [98]:

len(cols)


45

In [108]:
impulse_response_lag_1 = pd.DataFrame(m.irfs[2, :, :], columns=colnames, index = colnames)

impulse_response_lag_1['SPY'].sort_values(ascending=False)

AMAT    0.001265
TGT     0.000279
MOS     0.000214
AIG    -0.000027
CTRA   -0.000078
MU     -0.000083
CSCO   -0.000094
SBUX   -0.000139
AMD    -0.000150
RRC    -0.000160
DIS    -0.000210
CLX    -0.000258
ADBE   -0.000306
DLTR   -0.000338
JPM    -0.000342
LMT    -0.000345
KO     -0.000366
OXY    -0.000374
K      -0.000387
PGR    -0.000486
CAG    -0.000487
AA     -0.000509
SONY   -0.000549
QCOM   -0.000551
WBA    -0.000570
EOG    -0.000578
CVX    -0.000646
IBM    -0.000654
PEP    -0.000781
X      -0.000787
ORCL   -0.000872
TSN    -0.000885
KR     -0.000898
MMM    -0.000911
F      -0.000966
GE     -0.000973
CVS    -0.001051
WFC    -0.001067
BA     -0.001179
WMT    -0.001222
INTC   -0.001276
XOM    -0.001346
MO     -0.002187
AFL    -0.002287
SPY    -0.114138
Name: SPY, dtype: float64

In [109]:
m.H.shape

(2025, 1035)

In [110]:
cum_cov_effect = m.cum_effect_cov()

In [111]:
cum_cov_effect.shape

(11, 2025, 2025)

In [112]:
45 * 45

2025